In [1]:
# imports
import pandas as pd
import requests
import os

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [2]:
# Store the API keys in windows system environmrnt -- for security purpose
FSQapikey = os.environ['FOURSQUARE_API_KEY']
YLPapikey = os.environ['Yelp_API_KEY']


In [3]:
#load city_bikes dataframe to use it in FSQ and Yelp API
df_bikes = pd.read_csv('bikes.csv', index_col= 0)
df_bikes.head()

,latitude,longitude,free_bikes
0,43.665269,-79.319796,16
1,43.671420,-79.445947,0
2,43.666224,-79.317693,21
3,43.653236,-79.376716,0
4,43.663722,-79.380288,8


In [7]:
# get all FSQ information for each bike station in Toronto city
results = []
headers = {
        "Accept": "application/json",
        "Authorization": FSQapikey
 }

for i in range(len(df_bikes)):
    #use lat and long from city_bikes dataframe
    lat = df_bikes.iloc[i,0] 
    long = df_bikes.iloc[i,1]
    url = f'https://api.foursquare.com/v3/places/search?ll={lat},{long}&radius=1000'
    response = requests.get(url, headers=headers)
    res_json = response.json()
    results.append(res_json) # put all the inforamtion in a list

In [72]:
#results

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [8]:
# no rating info availabe in FSQ API
fsq_data = []
for i in range(len(results)):
    for j in range(len(results[i]['results'])):
        try:    
            POI = results[i]['results'][j]['categories'][0]['name']
            lat = results[i]['results'][j]['geocodes']['main']['latitude']
            long = results[i]['results'][j]['geocodes']['main']['longitude']
            address = results[i]['results'][j]['location']['formatted_address']
            city = results[i]['results'][j]['location']['locality']
            street = results[i]['results'][j]['location']['cross_street']
            postCode = results[i]['results'][j]['location']['postcode']
        except:
            continue
        fsq_dict = {'POI':POI, 'latitude':lat, 'longitude':long, 'address':address, 'city':city, 'postCode':postCode}
        fsq_data.append(fsq_dict)

Put your parsed results into a DataFrame

In [73]:
fsq_df = pd.DataFrame(fsq_data)
fsq_df.head()

,POI,latitude,longitude,address,city,postCode
0,Brewery,43.663372,-79.319902,"1001 Eastern Ave (Woodfield Rd), Toronto ON M4...",Toronto,M4L 1A8
1,Diner,43.665121,-79.321347,"1483 Queen St E, Toronto ON M4L 1E2",Toronto,M4L 1E2
2,Bicycle Store,43.666157,-79.317863,"1592 Queen St E, Toronto ON M4L 1G1",Toronto,M4L 1G1
3,Physician,43.665980,-79.318770,"1566 Queen St E (at Ashdale Ave.), Toronto ON ...",Toronto,M4L 1E9
4,Park,43.662892,-79.312532,"Queen St (at Kingston Rd), Toronto ON M4L 1G7",Toronto,M4L 1G7


In [10]:
#export dataframe to csv file
fsq_df.to_csv('fsq.csv')

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [14]:
# repeat same steps above
yelpResults = []
yelpHeaders = {
        "Accept": "application/json",
        "Authorization": YLPapikey
 }
for i in range(len(df_bikes)):
    #use lat and long from city_bikes dataframe
    lat = df_bikes.iloc[i,0] 
    long = df_bikes.iloc[i,1]
    url = f'https://api.yelp.com/v3/businesses/search?latitude={lat}&longitude={long}&radius=1000'
    response = requests.get(url, headers=yelpHeaders)
    res_json = response.json()
    yelpResults.append(res_json) # put all the inforamtion in a list

In [74]:
#yelpResults

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [55]:
yelp_data = []
for i in range(500): # Yelp api limited to 500 records
        for j in range(len(yelpResults[i]['businesses'])): 
            try:
                POI = yelpResults[i]['businesses'][j]['name']
                category1 = yelpResults[i]['businesses'][j]['categories'][0]['alias']
                category2 = yelpResults[i]['businesses'][j]['categories'][0]['title']
                rating = yelpResults[i]['businesses'][j]['rating']
                lat = yelpResults[i]['businesses'][j]['coordinates']['latitude']
                long = yelpResults[i]['businesses'][j]['coordinates']['longitude']
                street = yelpResults[i]['businesses'][j]['location']['address1']
                city = yelpResults[i]['businesses'][j]['location']['city']
                postCode = yelpResults[i]['businesses'][j]['location']['zip_code']
                price = yelpResults[i]['businesses'][j]['price']
            except:
                continue
            yelp_dict = {'POI':POI, 'rating':rating,'category1':category1,'category2':category2, 'latitude':lat, 'longitude':long, 'price':price, 'city':city, 'street':street, 'postCode':postCode}
            yelp_data.append(yelp_dict)

Put your parsed results into a DataFrame

In [76]:
yelp_df = pd.DataFrame(yelp_data)
yelp_df.head()

,POI,rating,category1,category2,latitude,longitude,price,city,street,postCode
0,The Burger's Priest,3.5,burgers,Burgers,43.666700,-79.315585,$$,Toronto,1636 Queen Street E,M4L 1G3
1,Lake Inez,4.5,bars,Bars,43.672340,-79.320640,$$$,Toronto,1471 Gerrard Street E,M4L 2A1
2,Maha's,4.0,egyptian,Egyptian,43.671670,-79.328530,$$,Toronto,226 Greenwood Avenue,M4L 2R2
3,O Sushi,4.0,sushi,Sushi Bars,43.666540,-79.316930,$$,Toronto,6 Coxwell Avenue,M4L 3A7
4,Burrito Bandidos,3.5,mexican,Mexican,43.666561,-79.316422,$$,Toronto,1614 Queen Street E,M4L 1G4


In [77]:
yelp_df.to_csv('yelp.csv') # export the result into csv file

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Although it's limited to 500 records I prefer yelp, becuase it provides the rating and price for each business.


Get the top 10 restaurants according to their rating

In [78]:
top10_df = yelp_df.sort_values(by=['rating'], ascending=False)
top10_df.head(10)

,POI,rating,category1,category2,latitude,longitude,price,city,street,postCode
5990,Golden Gecko Coffee,5.0,coffee,Coffee & Tea,43.655960,-79.487420,$,Toronto,282 Jane Street,M6S 3Z2
4525,The Coffee Boutique,5.0,cafes,Cafes,43.650402,-79.478856,$,Toronto,2285 Booor Street W,M6S 1P1
6299,New Orleans Seafood & Steakhouse,5.0,seafood,Seafood,43.677750,-79.506110,$$$,Toronto,267 Scarlett Road,M6N 4L1
7721,Aroma Espresso Bar,5.0,coffee,Coffee & Tea,43.687291,-79.393605,$$,Toronto,1407 Yonge Street,M4T 1Y7
7758,COBS Bread,5.0,bakeries,Bakeries,43.651403,-79.475677,$$,Toronto,2204 Bloor Street W,M6S 1N4
6043,The Coffee Boutique,5.0,cafes,Cafes,43.650402,-79.478856,$,Toronto,2285 Booor Street W,M6S 1P1
4407,Papyrus,5.0,egyptian,Egyptian,43.677090,-79.353240,$$,Toronto,337 Danforth Avenue,M4K 1N7
6311,Tim Horton,5.0,coffee,Coffee & Tea,43.771677,-79.500777,$,Toronto,70 The Pond Road,M3J 3M6
4144,COBS Bread,5.0,bakeries,Bakeries,43.651403,-79.475677,$$,Toronto,2204 Bloor Street W,M6S 1N4
1581,Papyrus,5.0,egyptian,Egyptian,43.677090,-79.353240,$$,Toronto,337 Danforth Avenue,M4K 1N7
